# 11-830 Computational Ethics for NLP
## Homework 1

In [103]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import os
from tqdm import tqdm

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/athiyadeviyani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/athiyadeviyani/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Preprocessing

## Load the data

In [104]:
train = pd.read_table('snli/snli_1.0_train.txt', delimiter = '\t')

In [3]:
train.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN


In [4]:
premises = pd.DataFrame()
premises["sentence"] = train["sentence1"]

hypotheses = pd.DataFrame()
hypotheses["sentence"] = train["sentence2"]

In [5]:
premises.head()

,sentence
0,A person on a horse jumps over a broken down a...
1,A person on a horse jumps over a broken down a...
2,A person on a horse jumps over a broken down a...
3,Children smiling and waving at camera
4,Children smiling and waving at camera


In [6]:
hypotheses.head()

,sentence
0,A person is training his horse for a competition.
1,"A person is at a diner, ordering an omelette."
2,"A person is outdoors, on a horse."
3,They are smiling at their parents
4,There are children present


In [14]:
my_file = open('identity_labels.txt', "r")
content = my_file.read()
identities = content.split('\n')
identities[:10]

['woman', 'women', 'man', 'men', 'girl', 'girls', 'boy', 'boys', 'she', 'he']

## Preprocess the data

Lowercase, remove stopwords and tokenize the data. Remove duplicates.

In [7]:
premises.drop_duplicates(inplace=True)
hypotheses.drop_duplicates(inplace=True)

In [8]:
premises.describe()

,sentence
count,150736
unique,150736
top,A person on a horse jumps over a broken down a...
freq,1


In [9]:
hypotheses.describe()

,sentence
count,480040
unique,480040
top,A person is training his horse for a competition.
freq,1


In [29]:
tokenizer = RegexpTokenizer('[a-z]\w+')

def preprocess(sentence):
    if isinstance(sentence, float):
        return []
    else:
        sentence = sentence.lower()
        tokenized = tokenizer.tokenize(sentence)
        return list(set([word for word in tokenized if word not in stop_words]))
    
preprocess('Person on a horse jumps over a broken down airplane.')

['jumps', 'broken', 'airplane', 'horse', 'person']

In [11]:
# This cell takes a while to run (~2 mins)

premises["sentence"] = premises["sentence"].map(preprocess)
hypotheses["sentence"] = hypotheses["sentence"].map(preprocess)

In [13]:
hypotheses.head(1), premises.head(1)

(                                 sentence
 0  [horse, training, competition, person],
                                    sentence
 0  [jumps, broken, airplane, horse, person])

## Get the word occurence dictionary for premise and hypothesis

In [16]:
premise_dict = {}
hyp_dict = {}

def fill_dict(source, target_dict):
    for sentence in tqdm(source.values):
        sentence = sentence[0]
        for word in sentence:
            target_dict[word] = target_dict.get(word, 0) + 1
            
fill_dict(premises, premise_dict)
fill_dict(hypotheses, hyp_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 480041/480041 [00:00<00:00, 585438.03it/s]


In [18]:
co_premise_dict = {}
co_hyp_dict = {}

def fill_co_dict(source, target_dict):
    for sentence in tqdm(source.values):
        sentence = sentence[0]
        for identity in identities:
            if identity in sentence:
                for word in sentence:
                    key = (identity, word)
                    target_dict[key] = target_dict.get(key, 0) + 1
                    
fill_co_dict(premises, co_premise_dict)
fill_co_dict(hypotheses, co_hyp_dict)

100%|█████████████████████████████████████████████████████████████████████████████████████| 480041/480041 [00:07<00:00, 67428.21it/s]


In [40]:
key = ('woman', 'man')
co_premise_dict[key]

4289

In [38]:
sorted_coprem = sorted(co_premise_dict.items(), key=lambda x: x[1], reverse=True)
[((x,y), z) for ((x,y),z) in sorted_coprem if x != y][:10]

[(('man', 'shirt'), 6610),
 (('man', 'wearing'), 4781),
 (('men', 'two'), 4756),
 (('woman', 'man'), 4289),
 (('man', 'woman'), 4289),
 (('man', 'black'), 3718),
 (('black', 'man'), 3718),
 (('man', 'white'), 3524),
 (('white', 'man'), 3524),
 (('man', 'blue'), 3508)]

In [39]:
sorted_cohyp = sorted(co_hyp_dict.items(), key=lambda x: x[1], reverse=True)
[((x,y), z) for ((x,y),z) in sorted_cohyp if x != y][:10]

[(('men', 'two'), 9304),
 (('woman', 'man'), 5924),
 (('man', 'woman'), 5924),
 (('man', 'wearing'), 4779),
 (('women', 'two'), 4681),
 (('man', 'sitting'), 3456),
 (('man', 'playing'), 3356),
 (('man', 'shirt'), 3185),
 (('man', 'young'), 2933),
 (('young', 'man'), 2933)]

In [80]:
co_premise_dict = {k:v for k,v in co_premise_dict.items() if v >= 10}
co_hyp_dict = {k:v for k,v in co_hyp_dict.items() if v >= 10}

## Calculte PMI

In [85]:
def get_pmi(wi, wj, source="premise"):
    if source == "premise":
        word_count_dic = premise_dict
        co_dict = co_premise_dict
        N = len(premises.values)
    else:
        word_count_dic = hyp_dict
        co_dict = co_hyp_dict
        N = len(hypotheses.values)
    
    c_wi = word_count_dic[wi]
    c_wj = word_count_dic[wj]
    c_wi_wj = co_dict[(wi, wj)]
    # print(c_wi, c_wj, c_wi_wj)
    
    pmi = np.log2( (N * c_wi_wj) / (c_wi * c_wj) )
    
    return pmi

In [86]:
get_pmi('woman', 'man', source="premise")

-0.2797887539473858

In [87]:
def get_pmi_all(source="premise"):
    if source == "premise":
        co_dict = co_premise_dict
    else:
        co_dict = co_hyp_dict
    
    pmi_dict = {}
    for (identity, word) in co_dict:
        key = (identity, word)
        pmi_dict[key] = get_pmi(identity, word, source)
        
    return pmi_dict

In [88]:
pmi_premise = get_pmi_all("premise")

In [89]:
pmi_hyp = get_pmi_all("hyp")

In [90]:
sorted_pmiprem = sorted(pmi_premise.items(), key=lambda x: x[1], reverse=True)
[((x,y), z) for ((x,y),z) in sorted_pmiprem if x != y][:10]

[(('senior', 'citizen'), 11.839094409326705),
 (('polish', 'nail'), 11.294773893102894),
 (('senior', 'citizens'), 10.536974795519523),
 (('german', 'shepherd'), 10.363160868083144),
 (('french', 'horn'), 9.430175019210814),
 (('father', 'son'), 7.974449683918122),
 (('son', 'father'), 7.974449683918122),
 (('straight', 'ahead'), 7.945365061793425),
 (('american', 'footballer'), 7.394309566653809),
 (('mother', 'daughter'), 7.252713288269991)]

In [91]:
sorted_pmihyp = sorted(pmi_hyp.items(), key=lambda x: x[1], reverse=True)
[((x,y), z) for ((x,y),z) in sorted_pmihyp if x != y][:10]

[(('german', 'shepherd'), 11.736773510385337),
 (('senior', 'citizen'), 11.73381530258773),
 (('german', 'shepard'), 11.68050529073859),
 (('polish', 'nail'), 11.517006558455712),
 (('buddhist', 'temple'), 11.099109405037627),
 (('senior', 'citizens'), 11.06815756784879),
 (('french', 'fries'), 11.065443183151753),
 (('gay', 'pride'), 10.717979996157254),
 (('gay', 'rights'), 9.70789117853367),
 (('french', 'horn'), 9.65040568387291)]

In [105]:
def get_n_highest(id_word, pmi_dict, n=10):
    id_list = [((x,y),z) for ((x,y),z) in pmi_dict.items() if x==id_word]
    sorted_list = sorted(id_list, key=lambda x: x[1], reverse=True)[:n]
    return sorted_list

In [106]:
get_n_highest('woman', pmi_premise)

[(('woman', 'woman'), 2.89069675686166),
 (('woman', 'mascara'), 2.89069675686166),
 (('woman', 'lipstick'), 2.5911364750027523),
 (('woman', 'veil'), 2.528126677476952),
 (('woman', 'sari'), 2.4545976420549867),
 (('woman', 'clothesline'), 2.443237779890439),
 (('woman', 'headscarf'), 2.440035347852095),
 (('woman', 'blouse'), 2.414668627693668),
 (('woman', 'purse'), 2.3848058271317027),
 (('woman', 'handbag'), 2.3236561641377667)]

In [107]:
get_n_highest('woman', pmi_hyp)

[(('woman', 'woman'), 3.2569960723468223),
 (('woman', 'weaves'), 2.93506797745946),
 (('woman', 'chevrolet'), 2.8419585730679784),
 (('woman', 'veil'), 2.7715692451765803),
 (('woman', 'bra'), 2.7264813556480423),
 (('woman', 'applies'), 2.702407220669185),
 (('woman', 'headscarf'), 2.6469425906628357),
 (('woman', 'shawl'), 2.6469425906628357),
 (('woman', 'handbag'), 2.5980329901818893),
 (('woman', 'lipstick'), 2.5741862482275213)]

In [108]:
get_n_highest('man', pmi_premise)

[(('man', 'man'), 1.9647533772668104),
 (('man', 'girlfriend'), 1.8867508652655374),
 (('man', 'mustached'), 1.8578381733502984),
 (('man', 'shaves'), 1.8578381733502984),
 (('man', 'breakdances'), 1.8492761598468743),
 (('man', 'necktie'), 1.8272498535168755),
 (('man', 'rearing'), 1.8272498535168755),
 (('man', 'balding'), 1.8162308525567223),
 (('man', 'beard'), 1.8093990743221224),
 (('man', 'bearded'), 1.7935667272605869)]

In [109]:
get_n_highest('man', pmi_hyp)

[(('man', 'man'), 2.339316274915878),
 (('man', 'beared'), 2.339316274915878),
 (('man', 'mustache'), 2.12358758386044),
 (('man', 'shaven'), 2.1079907288094217),
 (('man', 'afro'), 2.1048510212788543),
 (('man', 'accepts'), 2.0983081754120825),
 (('man', 'propose'), 2.0983081754120825),
 (('man', 'bearded'), 2.057903339767229),
 (('man', 'hardhat'), 2.0344616933874566),
 (('man', 'balding'), 1.9673474975289196)]

# Analysis

In [111]:
identities

['woman',
 'women',
 'man',
 'men',
 'girl',
 'girls',
 'boy',
 'boys',
 'she',
 'he',
 'her',
 'him',
 'his',
 'female',
 'male',
 'mother',
 'father',
 'sister',
 'brother',
 'daughter',
 'son',
 'feminine',
 'masculine',
 'androgynous',
 'trans',
 'transgender',
 'transsexual',
 'nonbinary',
 'non-binary',
 'two-spirit',
 'hijra',
 'genderqueer',
 'black',
 'asian',
 'hispanic',
 'white',
 'african',
 'american',
 'latino',
 'latina',
 'caucasian',
 'africans',
 'middle-eastern',
 'australian',
 'australians',
 'asians',
 'european',
 'europeans',
 'chinese',
 'indian',
 'indonesian',
 'brazilian',
 'pakistani',
 'bangladeshi',
 'russian',
 'nigerian',
 'japanese',
 'mexican',
 'filipino ',
 'vietnamese ',
 'german',
 'egyptian',
 'ethiopian',
 'turkish',
 'iranian',
 'thai',
 'congolese',
 'french',
 'british ',
 'italian',
 'korean',
 'burmese',
 'canadian ',
 'australian ',
 'spanish',
 'dutch',
 'swiss',
 'saudi',
 'argentinian ',
 'taiwanese ',
 'swedish ',
 'belgian',
 'polish

In [112]:
# TODO: get 'womanly' and 'manly' words